In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [95]:
import pandas as pd
import numpy as np

from pydataset import data
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, round
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.functions import col, expr

In [31]:
print(spark)
print(spark.version)

3.0.1


# Exercises

### 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be `language`
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [3]:
lang_df = pd.DataFrame({"language": ['python', 'spark', 'sql', 'swift', 'javascript']})

In [4]:
lang_df

,language
0,python
1,spark
2,sql
3,swift
4,javascript


In [5]:
sp_df = spark.createDataFrame(lang_df)
sp_df

DataFrame[language: string]

In [6]:
sp_df.printSchema()

root
 |-- language: string (nullable = true)



In [7]:
#printing the shape of the spark df:
print(sp_df.count(), len(sp_df.columns))

5 1


In [8]:
sp_df.show(5)

+----------+
|  language|
+----------+
|    python|
|     spark|
|       sql|
|     swift|
|javascript|
+----------+



### 2 Load the mpg dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below:

`The 1999 audi a4 has a 4 cylinder engine. For each vehicle.`

b. Transform the trans column so that it only contains either manual or auto.

In [9]:
from pyspark.sql.functions import round, concat, sum, min, max, count, avg, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

In [20]:
# importing the data:
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [13]:
# Use lit

mpg.select(concat(lit("The "), mpg.year, lit(" "), mpg.manufacturer, lit(" "), mpg.model, lit(" has a "), mpg.cyl, lit(" engine")).alias("cylinders")).show(1)

+--------------------+
|           cylinders|
+--------------------+
|The 1999 audi a4 ...|
+--------------------+
only showing top 1 row



In [14]:
mpg.select(mpg.trans, when(mpg.trans.contains("auto"), "auto").otherwise("manual").alias("Trans")).show(5)

+----------+------+
|     trans| Trans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 5 rows



In [ ]:
# Testing out how to use the regex for this problem. Although I think I'll just end up using basic python usage:



### 3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [4]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [26]:
tips.count()

244

In [32]:
# So this is the count of the smoker column, just need to make it %
tips.groupBy('smoker').agg(count(tips.smoker)).show()

+------+-------------+
|smoker|count(smoker)|
+------+-------------+
|    No|          151|
|   Yes|           93|
+------+-------------+



In [33]:
# % of observations that are smokers:

tips.groupBy('smoker').agg(round(count(tips.smoker)/ tips.count(),2)).show()

+------+-------------------------------+
|smoker|round((count(smoker) / 244), 2)|
+------+-------------------------------+
|    No|                           0.62|
|   Yes|                           0.38|
+------+-------------------------------+



#### B. Create a column that contains the tip percentage

In [41]:
# This is accomplished by dividing the tip / total_bill by row...

tips.select(tips.total_bill, tips.tip).show(5)

+----------+----+
|total_bill| tip|
+----------+----+
|     16.99|1.01|
|     10.34|1.66|
|     21.01| 3.5|
|     23.68|3.31|
|     24.59|3.61|
+----------+----+
only showing top 5 rows



In [51]:
# Now to divide so it's a percentage:

tips.select(tips.total_bill, tips.tip, round(tips.total_bill / tips.tip, 2).alias('tip_percentage')).show()

+----------+----+--------------+
|total_bill| tip|tip_percentage|
+----------+----+--------------+
|     16.99|1.01|         16.82|
|     10.34|1.66|          6.23|
|     21.01| 3.5|           6.0|
|     23.68|3.31|          7.15|
|     24.59|3.61|          6.81|
|     25.29|4.71|          5.37|
|      8.77| 2.0|          4.39|
|     26.88|3.12|          8.62|
|     15.04|1.96|          7.67|
|     14.78|3.23|          4.58|
|     10.27|1.71|          6.01|
|     35.26| 5.0|          7.05|
|     15.42|1.57|          9.82|
|     18.43| 3.0|          6.14|
|     14.83|3.02|          4.91|
|     21.58|3.92|          5.51|
|     10.33|1.67|          6.19|
|     16.29|3.71|          4.39|
|     16.97| 3.5|          4.85|
|     20.65|3.35|          6.16|
+----------+----+--------------+
only showing top 20 rows



#### C. Calculate the average tip percentage for each combination of sex and smoker.

- First need to aggregate so that I have 4 categories that look like this:

Female   smoker

male     smoker

female   non-smoker

male    smoker

And in that way I can create something that then does the math for each of those columns... in other words I'll need to classify each row by aggregation, and then divide the tip by the total bill for each of those groups.

In [81]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [84]:
tips.groupBy('smoker', 'sex', 'total_bill', 'tip').agg(round(tips.tip / tips.total_bill, 2).alias("tip_percentage")).show(5)

+------+------+----------+----+--------------+
|smoker|   sex|total_bill| tip|tip_percentage|
+------+------+----------+----+--------------+
|    No|  Male|      21.5| 3.5|          0.16|
|   Yes|Female|     43.11| 5.0|          0.12|
|    No|Female|     16.93|3.07|          0.18|
|   Yes|  Male|     15.48|2.02|          0.13|
|    No|Female|     22.75|3.25|          0.14|
+------+------+----------+----+--------------+
only showing top 5 rows



In [86]:
# I think the above isn't quite right, coming back to in once I've finished the other assignments. I think I'm supposed to use expr somehow.

In [100]:
tip_percentage = tips.withColumn("tip_percentage", (round(col('tip')/col('total_bill'), 2)))

In [101]:
tip_percentage.show()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          0.06|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          0.16|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          0.17|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          0.14|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          0.15|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          0.19|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          0.23|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          0.12|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          0.13|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          0.22|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|          0.17|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|          0.14|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

In [102]:
tips.show()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|           6.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          16.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          17.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          14.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          15.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          19.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          23.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          12.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          13.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          22.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|          17.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|          14.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

In [103]:
tips.groupBy('smoker', 'sex').agg()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|           6.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          16.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          17.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          14.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          15.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          19.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          23.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          12.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          13.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          22.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|          17.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|          14.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to farenheight.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [87]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



#### Scratch area

In [20]:
mpg.select(sum(mpg.hwy)/count(mpg.hwy).alias("avg_1"), avg(mpg.hwy).alias("avg_2")).show()

+----------------------------------+-----------------+
|(sum(hwy) / count(hwy) AS `avg_1`)|            avg_2|
+----------------------------------+-----------------+
|                 23.44017094017094|23.44017094017094|
+----------------------------------+-----------------+



In [23]:
mpg.select(concat(mpg.manufacturer, lit(" "), lit(mpg.model))).show(5)

+------------------------------+
|concat(manufacturer,  , model)|
+------------------------------+
|                       audi a4|
|                       audi a4|
|                       audi a4|
|                       audi a4|
|                       audi a4|
+------------------------------+
only showing top 5 rows



In [21]:
mpg.select(sum(mpg.hwy) / count(mpg.hwy).alias("average_1")).show()

+--------------------------------------+
|(sum(hwy) / count(hwy) AS `average_1`)|
+--------------------------------------+
|                     23.44017094017094|
+--------------------------------------+



In [24]:
mpg.select(sum(mpg.hwy)).show()

+--------+
|sum(hwy)|
+--------+
|    5485|
+--------+

